In [1]:
# Install necessary packages
# Run the following commands to install the required libraries:
!pip install geopandas shapely

In [2]:
import geopandas as gpd
import requests


In [3]:
# Provide the STAC and RASTER API endpoints
# The endpoint is referring to a location within the API that executes a request on a data collection nesting on the server.

# The STAC API is a catalog of all the existing data collections that are stored in the GHG Center.
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"

# The RASTER API is used to fetch collections for visualization
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
# Name of the collection for ODIAC dataset
collection_name = "odiac-ffco2-monthgrid-v2023"

In [4]:
# Read the state boundaries directly from the URL
url = 'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip'
states_gdf = gpd.read_file(url)


In [5]:
# Transform CRS to WGS84 if not already
states_gdf = states_gdf.to_crs(epsg=4326)



In [6]:
# texas_aoi = {
#     "type": "Feature", # Create a feature object
#     "properties": {},
#     "geometry": { # Set the bounding coordinates for the polygon
#         "coordinates": [
#             [
#                 # [13.686159004559698, -21.700046934333145],
#                 # [13.686159004559698, -23.241974326585833],
#                 # [14.753560168039911, -23.241974326585833],
#                 # [14.753560168039911, -21.700046934333145],
#                 # [13.686159004559698, -21.700046934333145],
#                 [-95, 29], # South-east bounding coordinate
#                 [-95, 33], # North-east bounding coordinate
#                 [-104,33], # North-west bounding coordinate
#                 [-104,29], # South-west bounding coordinate
#                 [-95, 29]  # South-east bounding coordinate (closing the polygon)
#             ]
#         ],
#         "type": "Polygon",
#     },
# }

In [7]:
# Initialize a dictionary to hold the AOIs
state_aois = {}

# Loop through each state
for idx, row in states_gdf.iterrows():
    state_name = row['NAME']  # Get the state name
    geometry = row.geometry

    # Get the bounding box of the state's geometry
    minx, miny, maxx, maxy = geometry.bounds

    # Create the bounding box coordinates
    bbox_coords = [
        [minx, miny],  # Lower-left corner (min longitude, min latitude)
        [minx, maxy],  # Upper-left corner (min longitude, max latitude)
        [maxx, maxy],  # Upper-right corner (max longitude, max latitude)
        [maxx, miny],  # Lower-right corner (max longitude, min latitude)
        [minx, miny],  # Closing the polygon by returning to the first point
    ]

    # Construct the AOI dictionary
    # state_aoi = {
    #     "type": "Feature",
    #     "properties": {"name": state_name},
    #     "geometry": {
    #         "type": "Polygon",
    #         "coordinates": [bbox_coords],
    #     },
    # }

    state_aoi = {
    "type": "Feature", # Create a feature object
    "properties": {},
    "geometry": { # Set the bounding coordinates for the polygon
        "coordinates": [
            bbox_coords
        ],
        "type": "Polygon",
    },
}

    # Add to the dictionary
    state_aois[state_name] = state_aoi


In [8]:
for state_name, aoi in state_aois.items():
    print(f"State: {state_name}")
    coords = aoi['geometry']['coordinates'][0]
    print("Bounding Coordinates:")
    for coord in coords:
        print(f"  {coord}")
    print()


State: Mississippi
Bounding Coordinates:
  [-91.65500899999999, 30.173942999999998]
  [-91.65500899999999, 34.996052]
  [-88.097888, 34.996052]
  [-88.097888, 30.173942999999998]
  [-91.65500899999999, 30.173942999999998]

State: North Carolina
Bounding Coordinates:
  [-84.32186899999999, 33.842316]
  [-84.32186899999999, 36.588117]
  [-75.460621, 36.588117]
  [-75.460621, 33.842316]
  [-84.32186899999999, 33.842316]

State: Oklahoma
Bounding Coordinates:
  [-103.002565, 33.615832999999995]
  [-103.002565, 37.002206]
  [-94.430662, 37.002206]
  [-94.430662, 33.615832999999995]
  [-103.002565, 33.615832999999995]

State: Virginia
Bounding Coordinates:
  [-83.675395, 36.540738]
  [-83.675395, 39.466012]
  [-75.242266, 39.466012]
  [-75.242266, 36.540738]
  [-83.675395, 36.540738]

State: West Virginia
Bounding Coordinates:
  [-82.644739, 37.201482999999996]
  [-82.644739, 40.638801]
  [-77.71951899999999, 40.638801]
  [-77.71951899999999, 37.201482999999996]
  [-82.644739, 37.20148299999

In [19]:
state_aois["Texas"]

{'type': 'Feature',
 'properties': {},
 'geometry': {'coordinates': [[[-106.645646, 25.837377],
    [-106.645646, 36.500704],
    [-93.508292, 36.500704],
    [-93.508292, 25.837377],
    [-106.645646, 25.837377]]],
  'type': 'Polygon'}}

In [20]:
texas_aoi = {
    "type": "Feature", # Create a feature object
    "properties": {},
    "geometry": { # Set the bounding coordinates for the polygon
        "coordinates": [
            [
                # [13.686159004559698, -21.700046934333145],
                # [13.686159004559698, -23.241974326585833],
                # [14.753560168039911, -23.241974326585833],
                # [14.753560168039911, -21.700046934333145],
                # [13.686159004559698, -21.700046934333145],
                [-95, 29], # South-east bounding coordinate
                [-95, 33], # North-east bounding coordinate
                [-104,33], # North-west bounding coordinate
                [-104,29], # South-west bounding coordinate
                [-95, 29]  # South-east bounding coordinate (closing the polygon)
            ]
        ],
        "type": "Polygon",
    },
}
texas_aoi

{'type': 'Feature',
 'properties': {},
 'geometry': {'coordinates': [[[-95, 29],
    [-95, 33],
    [-104, 33],
    [-104, 29],
    [-95, 29]]],
  'type': 'Polygon'}}

In [10]:
# import geopandas as gpd
# import matplotlib.pyplot as plt
# from shapely.geometry import Polygon

# # Load Texas geometry
# url = 'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip'
# states_gdf = gpd.read_file(url)
# states_gdf = states_gdf.to_crs(epsg=4326)
# texas_gdf = states_gdf[states_gdf['NAME'] == 'Texas']

# # Original approximate bounding box from initial script
# approx_bbox_coords = [
#     [-95, 29],
#     [-95, 33],
#     [-104, 33],
#     [-104, 29],
#     [-95, 29],
# ]
# approx_bbox_polygon = Polygon(approx_bbox_coords)

# # Generated actual bounding box from script
# minx, miny, maxx, maxy = texas_gdf.geometry.total_bounds
# actual_bbox_coords = [
#     [minx, miny],
#     [minx, maxy],
#     [maxx, maxy],
#     [maxx, miny],
#     [minx, miny],
# ]
# actual_bbox_polygon = Polygon(actual_bbox_coords)

# # Create GeoDataFrames for bounding boxes
# approx_bbox_gdf = gpd.GeoDataFrame({'geometry': [approx_bbox_polygon]}, crs='EPSG:4326')
# actual_bbox_gdf = gpd.GeoDataFrame({'geometry': [actual_bbox_polygon]}, crs='EPSG:4326')

# # Plotting
# fig, ax = plt.subplots(figsize=(10, 8))

# # Plot Texas
# texas_gdf.plot(ax=ax, color='lightgray', edgecolor='black')

# # Plot approximate bounding box
# approx_bbox_gdf.plot(ax=ax, edgecolor='blue', facecolor='none', linewidth=2, label='Approximate Bounding Box')

# # Plot actual bounding box
# actual_bbox_gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2, label='Actual Bounding Box')

# plt.title('Comparison of Bounding Boxes for Texas')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.legend()
# plt.show()


In [11]:
# import geopandas as gpd

# # Load U.S. State Boundaries
# url = 'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip'
# states_gdf = gpd.read_file(url)

# # Ensure CRS is WGS84
# states_gdf = states_gdf.to_crs(epsg=4326)

# # Initialize dictionary to hold AOIs
# state_aois = {}

# # Loop through each state to create bounding box AOIs
# for idx, row in states_gdf.iterrows():
#     state_name = row['NAME']
#     geometry = row.geometry

#     # Get bounding box
#     minx, miny, maxx, maxy = geometry.bounds

#     # Create bounding box coordinates
#     bbox_coords = [
#         [minx, miny],  # Lower-left
#         [minx, maxy],  # Upper-left
#         [maxx, maxy],  # Upper-right
#         [maxx, miny],  # Lower-right
#         [minx, miny],  # Close polygon
#     ]

#     # Create AOI dictionary
#     state_aoi = {
#         "type": "Feature",
#         "properties": {"name": state_name},
#         "geometry": {
#             "type": "Polygon",
#             "coordinates": [bbox_coords],
#         },
#     }

#     state_aois[state_name] = state_aoi


In [12]:
# Create a function that would search for a data collection in the US GHG Center STAC API

# First, we need to define the function
# The name of the function = "get_item_count"
# The argument that will be passed through the defined function = "collection_id"
def get_item_count(collection_id):

    # Set a counter for the number of items existing in the collection
    count = 0

    # Define the path to retrieve the granules (items) of the collection of interest in the STAC API
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    # Run a while loop to make HTTP requests until there are no more URLs associated with the collection in the STAC API
    while True:

        # Retrieve information about the granules by sending a "get" request to the STAC API using the defined collection path
        response = requests.get(items_url)

        # If the items do not exist, print an error message and quit the loop
        if not response.ok:
            print("error getting items")
            exit()

        # Return the results of the HTTP response as JSON
        stac = response.json()

        # Increase the "count" by the number of items (granules) returned in the response
        count += int(stac["context"].get("returned", 0))

        # Retrieve information about the next URL associated with the collection in the STAC API (if applicable)
        next = [link for link in stac["links"] if link["rel"] == "next"]

        # Exit the loop if there are no other URLs
        if not next:
            break

        # Ensure the information gathered by other STAC API links associated with the collection are added to the original path
        # "href" is the identifier for each of the tiles stored in the STAC API
        items_url = next[0]["href"]

    # Return the information about the total number of granules found associated with the collection
    return count

In [13]:
number_of_items = get_item_count(collection_name)

In [14]:
number_of_items

276

In [15]:
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]

In [16]:
# Examine the first item in the collection
# Keep in mind that a list starts from 0, 1, 2... therefore items[0] is referring to the first item in the list/collection
items[0]

{'id': 'odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212',
 'bbox': [-180.0, -90.0, 180.0, 90.0],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023/items/odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212'},
  {'title': 'Map of Item',
   'href': 'https://earth.gov/ghgcenter/api/raster/collections/odiac-ffco2-monthgrid-v2023/items/odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212/map?assets=co2-emissions&rescale=-10%2C60&colormap_name=jet',
   'rel': 'previ

In [17]:
# Now we create a dictionary where the start datetime values for each granule is queried more explicitly by year and month (e.g., 2020-02)
items = {item["properties"]["start_datetime"][:7]: item for item in items}

# Next, we need to specify the asset name for this collection
# The asset name is referring to the raster band containing the pixel values for the parameter of interest
# For the case of the ODIAC Fossil Fuel CO₂ Emissions collection, the parameter of interest is “co2-emissions”
asset_name = "co2-emissions"

In [21]:
# Fetching the min and max values for a specific item
rescale_values = {"max":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}

In [22]:
rescale_values

{'max': 31415.447265625, 'min': -675.1028442382812}

In [23]:
items['2020-01']['collection'], items['2020-01']['id']

('odiac-ffco2-monthgrid-v2023',
 'odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202001')

In [ ]:
# # Texas, USA
# texas_aoi = {
#     "type": "Feature", # Create a feature object
#     "properties": {},
#     "geometry": { # Set the bounding coordinates for the polygon
#         "coordinates": [
#             [
#                 # [13.686159004559698, -21.700046934333145],
#                 # [13.686159004559698, -23.241974326585833],
#                 # [14.753560168039911, -23.241974326585833],
#                 # [14.753560168039911, -21.700046934333145],
#                 # [13.686159004559698, -21.700046934333145],
#                 [-95, 29], # South-east bounding coordinate
#                 [-95, 33], # North-east bounding coordinate
#                 [-104,33], # North-west bounding coordinate
#                 [-104,29], # South-west bounding coordinate
#                 [-95, 29]  # South-east bounding coordinate (closing the polygon)
#             ]
#         ],
#         "type": "Polygon",
#     },
# }

# '''
# State: Texas
# Bounding Coordinates:
#   [-106.645646, 25.837377]
#   [-106.645646, 36.500704]
#   [-93.508292, 36.500704]
#   [-93.508292, 25.837377]
#   [-106.645646, 25.837377]

# state_aoi = {
#         "type": "Feature",
#         "properties": {"name": state_name},
#         "geometry": {
#             "type": "Polygon",
#             "coordinates": [bbox_coords],
#         },
#     }

#     state_aois[state_name] = state_aoi


# ''';

In [24]:
# Check total number of items available within the collection
items = requests.get(
    f"{STAC_API_URL}/collections/{collection_name}/items?limit=300"
).json()["features"]

# Print the total number of items (granules) found
print(f"Found {len(items)} items")

Found 276 items


In [25]:
# Examine the first item in the collection
items[0]

{'id': 'odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212',
 'bbox': [-180.0, -90.0, 180.0, 90.0],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/odiac-ffco2-monthgrid-v2023/items/odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212'},
  {'title': 'Map of Item',
   'href': 'https://earth.gov/ghgcenter/api/raster/collections/odiac-ffco2-monthgrid-v2023/items/odiac-ffco2-monthgrid-v2023-odiac2023_1km_excl_intl_202212/map?assets=co2-emissions&rescale=-10%2C60&colormap_name=jet',
   'rel': 'previ

In [37]:
# The bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
# Create a function that retrieves information regarding a specific granule using its asset name and raster identifier and generates the statistics for it

# The function takes an item (granule) and a JSON (polygon) as input parameters
def generate_stats(item, geojson):

    # A POST request is made to submit the data associated with the item of interest (specific observation) within the boundaries of the polygon to compute its statistics
    result = requests.post(

        # Raster API Endpoint for computing statistics
        f"{RASTER_API_URL}/cog/statistics",

        # Pass the URL to the item, asset name, and raster identifier as parameters
        params={"url": item["assets"][asset_name]["href"]},

        # Send the GeoJSON object (polygon) along with the request
        json=geojson,

    # Return the response in JSON format
    ).json()

    # Return a dictionary containing the computed statistics along with the item's datetime information.
    return {
        **result["properties"],
        "start_datetime": item["properties"]["start_datetime"][:7],
    }

In [38]:
%%time
# %%time = Wall time (execution time) for running the code below

# Generate statistics using the created function "generate_stats" within the bounding box defined by the polygon
# This process may take a few minutes to complete
stats = [generate_stats(item, texas_aoi) for item in items]

CPU times: user 2.68 s, sys: 253 ms, total: 2.94 s
Wall time: 3min 52s


In [39]:
# Create a function that converts statistics in JSON format into a pandas DataFrame
import pandas as pd
def clean_stats(stats_json) -> pd.DataFrame:

    # Normalize the JSON data
    df = pd.json_normalize(stats_json)

    # Replace the naming "statistics.b1" in the columns
    df.columns = [col.replace("statistics.b1.", "") for col in df.columns]

    # Set the datetime format
    df["date"] = pd.to_datetime(df["start_datetime"])

    # Return the cleaned format
    return df

# Apply the generated function on the stats data
df = clean_stats(stats)

# Display the stats for the first 5 granules in the collection in the table
# Change the value in the parenthesis to show more or a smaller number of rows in the table
df.head(5)

,start_datetime,min,max,mean,count,sum,std,median,majority,minority,unique,histogram,valid_percent,masked_pixels,valid_pixels,percentile_2,percentile_98,date
0,2022-12,0.743868,420840.18750,39.447887,177466.0,7000659.0,1908.116283,3.214699,64.481010,0.743868,160218.0,"[[177450.0, 9.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0,...",34.23,340934.0,177466.0,1.317880,220.576385,2022-12-01
1,2022-11,0.608937,343526.96875,32.262497,177466.0,5725496.5,1557.576964,2.635552,52.784710,0.608937,160204.0,"[[177450.0, 9.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0,...",34.23,340934.0,177466.0,1.081165,180.777283,2022-11-01
2,2022-10,0.617746,348376.68750,32.727840,177466.0,5808079.0,1579.566634,2.674915,53.548267,0.617746,160216.0,"[[177450.0, 9.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0,...",34.23,340934.0,177466.0,1.097266,183.435852,2022-10-01
3,2022-09,0.737875,417608.00000,39.143070,177466.0,6946564.0,1893.460654,3.190599,63.961494,0.737875,160221.0,"[[177450.0, 9.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0,...",34.23,340934.0,177466.0,1.308810,219.010132,2022-09-01
4,2022-08,0.910771,516733.53125,48.361969,177466.0,8582605.0,2342.897245,3.935843,78.948669,0.910771,160211.0,"[[177450.0, 9.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0,...",34.23,340934.0,177466.0,1.613806,270.067291,2022-08-01


In [ ]:
state_stats = {}

for state_name, aoi in state_aois.items():
    print(f"Generating stats for {state_name}...")
    stats_list = []
    for item in items:
        try:
            stats = generate_stats(item, aoi)
            if stats is not None:
                stats_list.append(stats)
        except Exception as e:
            print(f"Error processing item {item.get('id', 'Unknown')} for {state_name}: {e}")
    state_stats[state_name] = stats_list

Generating stats for Mississippi...
Generating stats for North Carolina...
Generating stats for Oklahoma...
Generating stats for Virginia...
